## 1. 뉴스제목 가져오기
* user-agent 요청헤더를 반드시 설정해야 한다.

In [1]:
# requests 라이브러리 설치여부 확인
!pip show requests

Name: requests
Version: 2.32.3
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache-2.0
Location: C:\Users\KimGyuBhin\anaconda3\Lib\site-packages
Requires: certifi, charset-normalizer, idna, urllib3
Required-by: anaconda-catalogs, anaconda-client, anaconda-cloud-auth, anaconda-project, conda, conda-build, conda-repo-cli, conda_package_streaming, cookiecutter, datashader, jupyterlab_server, panel, requests-file, requests-toolbelt, Sphinx, streamlit, tldextract


In [3]:
# beautifulsoup4 라이브러리 설치여부 확인
!pip show beautifulsoup4

Name: beautifulsoup4
Version: 4.12.3
Summary: Screen-scraping library
Home-page: https://www.crummy.com/software/BeautifulSoup/bs4/
Author: 
Author-email: Leonard Richardson <leonardr@segfault.org>
License: MIT License
Location: C:\Users\KimGyuBhin\anaconda3\Lib\site-packages
Requires: soupsieve
Required-by: conda-build, nbconvert


In [1]:
# reqeusts, bs4 import
# BeautifulSoup 클래스 import
import requests
import bs4
from bs4 import BeautifulSoup

In [10]:
# requests, bs4 버전 확인하기
print(f'requests 버전 = {requests.__version__}')
print(f'bs4 버전 = {bs4.__version__}')


requests 버전 = 2.32.3
bs4 버전 = 4.12.3


### 1. 뉴스 제목 추출하기

In [ ]:
req_param = {
    'sid': 105
}

#sid = 104
#url = f'https://news.naver.com/section/{sid}'

url = 'https://news.naver.com/section/{sid}'.format(**req_param)
print(url)

https://news.naver.com/section/105


In [4]:
# IT/과학 뉴스 
req_param = {
    'sid': 105
}

url = 'https://news.naver.com/section/{sid}'.format(**req_param)
print(url)

# 요청 헤더 설정 : 브라우저 정보
req_header = {
    "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Whale/4.32.315.22 Safari/537.36"
}

# requests 의 get() 함수 호출하기 
res = requests.get(url, headers=req_header)
print(res.status_code, res.ok)
print(type(res))
print(res)

# 응답(response)이 OK(200) 이면
# 응답 (response)에서 text 추출
if res.ok:
    html = res.text
    #print(html)
    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(html, 'html.parser')
    #print(soup)
    
    # CSS 선택자
    # print(soup.select("div.sa_text a[href*='mnews/article']"))
    '''
    <div class='sa_text'>
        <a href='https://n.news.naver.com/mnews/article/082/0001336598'>
    '''
    a_tags = soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/article'].sa_text_title")
    print(len(a_tags))
    print(type(a_tags)) #[Tag,Tag,Tag]
    # <a> 태그 리스트 순회하기
    for idx,a_tag in enumerate(a_tags,1):        
        if a_tag.text:
            title = a_tag.text.strip() # 제목
            link = a_tag['href'] # 링크
            #print(len(title))
            print(f'{idx} {title} {link}')
else:
    # 응답(response)이 Error 이면 status code 출력    
    print(f'Error Code = {res.status_code}')

https://news.naver.com/section/105
200 True
<class 'requests.models.Response'>
<Response [200]>
42
<class 'bs4.element.ResultSet'>
1 '갤럭시Z 플립7·폴드7' 사전판매 104만대… 역대 최대 https://n.news.naver.com/mnews/article/015/0005161123
2 삼성 폴더블 점유율 10% 하락 전망…"타 브랜드와 경쟁 심화" https://n.news.naver.com/mnews/article/277/0005626428
3 솔트룩스, KT ‘믿음’과 손잡아…‘독자 AI’ 사업 참전 https://n.news.naver.com/mnews/article/009/0005529262
4 최휘영 문체장관 후보자 "게임이용장애 질병화 신중해야" https://n.news.naver.com/mnews/article/001/0015521296
5 만화계, 日 귀화 '뉴토끼' 운영자 송환 서명 운동 https://n.news.naver.com/mnews/article/030/0003334160
6 네이버, 경량 추론 AI 무료 공개…"학습 비용 동급 모델 100분의 1" https://n.news.naver.com/mnews/article/015/0005161259
7 “질 높은 AI 원료 구하라”… 이젠 데이터 확보 전쟁 https://n.news.naver.com/mnews/article/023/0003918769
8 인간처럼 사고하고 증명하고 문제 해결...수학올림피아드 금메달 딴 ‘구글 AI’ https://n.news.naver.com/mnews/article/009/0005529510
9 단통법 폐지 첫날 보조금 전쟁? 휴대폰 성지 아직은 '잠잠' https://n.news.naver.com/mnews/article/629/0000409765
10 폴라리스오피스, 핸디소프트와 '한컴 텃밭' 공공 시장 노린다 https://n.news

### 1.1 뉴스제목 추출하는 함수 선언하기

In [5]:

import requests
from bs4 import BeautifulSoup

#section_dict = {100:'정치',101:'경제',102:'사회',103:'생활/문화',104:'세계',105:'IT/과학'}
section_dict = {'정치':100,'경제':101,'사회':102,'생활/문화':103,'세계':104,'IT/과학':105}

def print_news(section):  #print_new(103,'생활/문화') 
    sid = section_dict[section]
    url = f'https://news.naver.com/section/{sid}'
    print(f'{url} {section} 뉴스')

    # 요청 헤더 설정 : 브라우저 정보
    req_header = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    }
    # requests 의 get() 함수 호출하기 
    res=requests.get(url,headers=req_header)
    print(res.status_code, res.ok)
    if res.ok:
        html = res.text
        # BeautifulSoup 객체 생성
        soup = BeautifulSoup(html, "html.parser")
        a_tags = soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/article'].sa_text_title")
        print(len(a_tags))
        # <a> 태그 리스트 순회하기
        for idx,a_tag in enumerate(a_tags,1):        
            if a_tag.text:
                title = a_tag.text.strip() # 제목
                link = a_tag['href'] # 링크
                #print(len(title))
                print(f'{idx} {title} {link}')
    else:
        # 응답(response)이 Error 이면 status code 출력    
        print(f'Error Code = {res.status_code}')       


In [6]:
print_news('세계')

https://news.naver.com/section/104 세계 뉴스
200 True
45
1 방글라데시 학교에 공군 훈련기 '추락'…최소 18명 사망 https://n.news.naver.com/mnews/article/277/0005625810
2 "비트코인은 사기"라던 JP모건 가상화폐 담보대출 출시 검토 https://n.news.naver.com/mnews/article/009/0005529428
3 트럼프 “워크 정책 반대” 유네스코 또 탈퇴…바이든 재가입 2년 만에 뒤집기 https://n.news.naver.com/mnews/article/022/0004053937
4 러·우크라 23일 튀르키예서 협상…양측 7주 만에 협상 테이블로 https://n.news.naver.com/mnews/article/448/0000544158
5 일본서 연락 끊겼던 20대 한국 여성 안전 확인 https://n.news.naver.com/mnews/article/005/0001791345
6 챗봇 인공지능이 수학 올림피아드 금메달 수준 도달 https://n.news.naver.com/mnews/article/003/0013376576
7 '파월 해임' 놓고 월가 혼란[투자의 창] https://n.news.naver.com/mnews/article/011/0004512229
8 구글 AI챗봇, 국제수학올림피아드 '금메달'…"서술형 답안 놀라워" https://n.news.naver.com/mnews/article/001/0015521508
9 오픈AI·소뱅의 '5000억 달러' 스타게이트, 쉽지 않네 [윤민혁의 실리콘밸리View] https://n.news.naver.com/mnews/article/011/0004511928
10 앞바다서 대규모 유전 발견…'에너지 독립' 기대하는 이 나라 https://n.news.naver.com/mnews/article/025/0003457001
11 [News&Why] 트럼프가 유럽 지켜주겠어? 똘똘 뭉치는 영·프·

### 2. Image 다운로드
* referer 요청 헤더를 반드시 설정해야 한다.

In [48]:

import requests
import os

req_header = {
    'referer':'https://comic.naver.com/webtoon/detail?titleId=817246&no=70&week=thu',
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"

}

img_urls = [
    'https://image-comic.pstatic.net/webtoon/817246/70/20241204175519_7983f60a778551c4c9a190394ff2e1f2_IMAG01_1.jpg',
    'https://image-comic.pstatic.net/webtoon/817246/70/20241204175519_7983f60a778551c4c9a190394ff2e1f2_IMAG01_2.jpg',
    'https://image-comic.pstatic.net/webtoon/817246/70/20241204175519_7983f60a778551c4c9a190394ff2e1f2_IMAG01_3.jpg'
]

for img_url in img_urls:
    # requests 의 get(url, headers) 함수 호출하기 
    res = requests.get(img_url,headers=req_header)
    print(res.ok, res.status_code)
    if res.ok:
        # binary 응답 데이터 가져오기
        img_data=res.content
        # print(img_data)
        # url에서 파일명만 추출하기
        file_name = os.path.basename(img_url)
        # binday data를 file에 write하기
        with open(file_name, 'wb') as file:            
            print(f'Writing to {file_name} ({len(img_data):,})')
            file.write(img_data)


True 200
Writing to 20241204175519_7983f60a778551c4c9a190394ff2e1f2_IMAG01_1.jpg (165,299)
True 200
Writing to 20241204175519_7983f60a778551c4c9a190394ff2e1f2_IMAG01_2.jpg (245,556)
True 200
Writing to 20241204175519_7983f60a778551c4c9a190394ff2e1f2_IMAG01_3.jpg (301,184)


* 현재 요청된 페이지의 image 모두 다운로드 해보기

In [ ]:
import requests
from bs4 import BeautifulSoup
import os

toon_url = 'https://comic.naver.com/webtoon/detail?titleId=834369&no=31&week=tue'

req_header={
    'referer':toon_url,
    'user-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
}

res = requests.get(toon_url)
if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    #img src의 끝이 .jpg인 수
    print("1번 시도 : ", len(soup.select("img[src$='.jpg']")))
    #img src의 시작이 https://image-comic.pstatic.net/webtoon인 수
    print("2번 시도 : ", len(soup.select("img[src^='https://image-comic.pstatic.net/webtoon']")))
    #img src에 _IMAG01이 포함되어 있는 수 => 정답 ~
    print("3번 시도 : ", len(soup.select("img[src*='IMAG01']")), "-> 정답")
    # img src file명을 저장할 빈 list 선언
    img_url_list = list()
    img_tags = soup.select("img[src*='IMAG01']")

    for img_tag in img_tags:
        img_url = img_tag['src']
        img_url_list.append(img_url)
    # print(len(img_url_list), img_url_list[:3])

    # img 디렉토리가 없으면 생성
    imgdir = 'img'
    if not os.path.isdir(imgdir): 
        os.mkdir(imgdir)
    
    for jpg_url in img_url_list:
        res = requests.get(jpg_url, headers=req_header)
        if res.ok:
            # binary data 추출하기
            img_data = res.content
            # img/xxx.jpg 디렉토리명과 jpg 파일명을 join
            dir_name = os.path.join(imgdir, os.path.basename(jpg_url))
            with open(dir_name, 'wb') as file:
                # print(f'Writing to {dir_name} ({len(img_data):,})')
                file.write(img_data)               
        else: print(f'Error Code : {res.status_code}')
    print("이미지 폴더 생성 완료 !")

1번 시도 :  126
2번 시도 :  126
3번 시도 :  94 -> 정답
이미지 폴더 생성 완료 !


### 2-1. Image 다운로드 코드 리팩토링

In [ ]:
# 기본 설정
url = 'https://comic.naver.com/webtoon/detail?titleId=833255&no=3&week=tue'
req_header = {'referer': url}
imgdir_name = 'img'

# 이미지 저장 폴더가 없으면 생성
os.makedirs(imgdir_name, exist_ok=True)

# 웹 페이지 요청 및 확인
res = requests.get(url)
if not res.ok:
    print(f'Error Code = {res.status_code}')
    exit()

# 이미지 URL 추출
soup = BeautifulSoup(res.text, 'html.parser')
img_url_list = [img_tag['src'] for img_tag in soup.select("img[src*='IMAG01']")]

# 이미지 다운로드
for img_url in img_url_list:
    res = requests.get(img_url, headers=req_header)
    if res.ok:
        img_data = res.content
        file_path = os.path.join(imgdir_name, os.path.basename(img_url))
        with open(file_path, 'wb') as file:
            print(f'Writing to {file_path} ({len(img_data):,} bytes)')
            file.write(img_data)
    else:
        print(f'Error Code = {res.status_code} for {img_url}')

### 3. 파일 업로드 하기
* http://httpbin.org/post 업로드 요청을 할 수 있는 url

In [30]:
import requests

# binary file 읽기
upload_files = {
    'img1': open('f1.jpg','rb'),
    'img2': open('f2.jpg','rb')
}
print(upload_files)

url = 'http://httpbin.org/post'
# file 업로드 하려면 requests의 post 함수에 files 속성을 사용합니다.
res = requests.post(url, files=upload_files)
print('연결 상태 : ', res.status_code)
print(res.json())
print(res.json()['files']['img1'])

{'img1': <_io.BufferedReader name='f1.jpg'>, 'img2': <_io.BufferedReader name='f2.jpg'>}
연결 상태 :  200
{'args': {}, 'data': '', 'files': {'img1': '...', 'img2': '...'}, 'form': {}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Content-Length': '411101', 'Content-Type': 'multipart/form-data; boundary=e2ca3b7f78796b5d1a0739ac2dea51ea', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.32.3', 'X-Amzn-Trace-Id': 'Root=1-6880460a-6a9ceb6625a1681764fdfbad'}, 'json': None, 'origin': '175.214.128.192', 'url': 'http://httpbin.org/post'}
...


### 4. 캡챠(이미지) API 호출하기
* urllib 사용
* 1. 캡차 키 발급 요청
  2. 캡차 이미지 요청
  3. 사용자 입력값 검증 요청

In [ ]:
# 캡차 키 발급 요청
import urllib.request

client_id = "@" # 개발자센터에서 발급받은 Client ID 값
client_secret = "@" # 개발자센터에서 발급받은 Client Secret 값

code = "0"
url = "https://openapi.naver.com/v1/captcha/nkey?code=" + code
#Request 객체생성
request = urllib.request.Request(url)
#요청헤더 설정
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
#서버에 request 전달
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{"key":"NyYLflKMhznOzjUV"}


In [ ]:
# 캡차 이미지 요청
import urllib.request

client_id = "@" # 개발자센터에서 발급받은 Client ID 값
client_secret = "@" # 개발자센터에서 발급받은 Client Secret 값
key = "@" # 캡차 Key 값
url = "https://openapi.naver.com/v1/captcha/ncaptcha.bin?key=" + key

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    print("캡차 이미지 저장")
    response_body = response.read()
    with open('captcha.jpg', 'wb') as f:
        f.write(response_body)
else:
    print("Error Code:" + rescode)

캡차 이미지 저장


In [ ]:
#  사용자 입력값 검증 요청
import urllib.request

client_id = "@" # 개발자센터에서 발급받은 Client ID 값
client_secret = "@" # 개발자센터에서 발급받은 Client Secret 값
code = "1"
key = "@"
value = "7T91"
url = "https://openapi.naver.com/v1/captcha/nkey?code=" + code + "&key=" + key + "&value=" + value
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)


{"result":false,"responseTime":48.73}


* requests를 사용하는 코드로 변경하기
* [requests docs](https://requests.readthedocs.io/en/latest/user/quickstart/)

In [ ]:
# 사용자 입력값 검증 요청
import os
from dotenv import load_dotenv
import requests

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수에서 값 읽기
client_id = "@" # 개발자센터에서 발급받은 Client ID 값
client_secret = "@" # 개발자센터에서 발급받은 Client Secret 값
key = "@"
#os.getenv("NAVER_CAPTCHA_KEY")  # .env 파일의 NAVER_CAPTCHA_KEY

# 사용자 입력값
code = "1"
value = "2N53S"  # 실제 사용시 사용자 입력값으로 대체 필요

# URL 구성
# query string key1=value1&key2=value2
url = f"https://openapi.naver.com/v1/captcha/nkey?code={code}&key={key}&value={value}"

# 요청 헤더
headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
}

# 요청 보내기
try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # 4xx, 5xx 오류 발생시 예외 발생
    
    # 응답 처리
    print(response.text)
    
except requests.exceptions.RequestException as e:
    print(f"Error occurred: {e}")
    print(f"Status Code: {response.status_code if 'response' in locals() else 'N/A'}")


Error occurred: 403 Client Error: Forbidden for url: https://openapi.naver.com/v1/captcha/nkey?code=1&key=NyYLflKMhznOzjUV&value=2N53S
Status Code: 403


### 5. 블로그 검색하기

In [ ]:
import requests
from pprint import pprint
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수에서 값 읽기
client_id = os.getenv("NAVER_CLIENT_ID")  # .env 파일의 NAVER_CLIENT_ID
client_secret = os.getenv("NAVER_CLIENT_SECRET")  # .env 파일의 NAVER_CLIENT_SECRET

headers = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret,
}

# query string 문자열을 dict 선언
payload = {
    'query': '파이썬',
    'display': 100,
    'sort': 'date'
}

url = 'https://openapi.naver.com/v1/search/blog.json' #?query=파이썬&display=100&sort=sim

# requests get(url, params, headers) 요청 
res = requests.get(url, params=payload, headers=headers)
# json() 함수로 응답 결과 가져오기
items_data = res.json()['items']
print(type(res.json()))
#pprint(res.json())
print(len(items_data), type(items_data), type(items_data[0]))
pprint(items_data[0])

#[{},{},{}] 
# [] - 100 개의 데이터, {title:'',link:''} - 1개의 블로그 글 데이터

In [ ]:
# 'title' , 'bloggername' , 'description' , 'bloggerlink' , 'link'
items_list = list()
item_list = []
for item in items_data:
#     print(item)
    item_list.append(item['title'])
    item_list.append(item['bloggername'])
    item_list.append(item['description'])
    item_list.append(item['bloggerlink'])
    item_list.append(item['link'])
    item_list.append(item['postdate'])

    items_list.append(item_list)
    item_list = []

print(items_list) #[[],[],[]]

# 'data/nhnblog.txt' 파일 생성하기
with open('data/nhnblog.txt','w',encoding="utf-8")as file:
    for items in items_list:
        for item in items:
            item = item + '\n'
            file.write(item)
        file.write('-'*150+'\n')